# Módulo 1 Utilización, procesamiento y visualización de grandes volúmenes de datos (Portafolio Análisis)

Oscar Eduardo Nieto Espitia - A01705090

 Se instala Java 8, descarga e instala Apache Spark, y configura las variables de entorno necesarias para que Python pueda utilizar Spark. Luego, se utiliza findspark para asegurarse de que Python pueda encontrar Spark. Esto te permitirá utilizar Spark en el entorno de trabajo de Python.

In [ ]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.4.1//spark-3.4.1-bin-hadoop3.tgz
!tar xf spark-3.4.1-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.1-bin-hadoop3"

import findspark
findspark.init()
findspark.find()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
22 packages can be upgraded. Run 'apt list --upgradable' to see them.


'/content/spark-3.4.1-bin-hadoop3'

## Dataset
El conjunto de datos utilizado en este proyecto contiene información sobre incidentes de crímenes en la ciudad de Chicago desde 2001 hasta la fecha actual, excluyendo los datos de los últimos siete días. Los datos se obtienen del sistema CLEAR (Citizen Law Enforcement Analysis and Reporting) del Departamento de Policía de Chicago. Para proteger la privacidad de las víctimas de crímenes, las direcciones se muestran solo a nivel de bloque, y no se identifican ubicaciones específicas. Es importante destacar que los datos pueden basarse en información preliminar proporcionada por las partes que informan los incidentes y pueden cambiar después de investigaciones adicionales.

## Objetivo:
El objetivo del proyecto es construir un modelo de aprendizaje automático que pueda predecir si un sospechoso involucrado en un incidente de crimen en Chicago fue arrestado o no. Para lograr esto, se utilizarán diversas características de los incidentes, como si son domésticos, la ubicación, el distrito, el año y otras, para entrenar un modelo de clasificación. La columna "Arrest" en el dataset se utilizará como la variable objetivo, donde "1" podría representar arresto y "0" no arresto.
https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-Present/ijzp-q8t2

Montar Google Drive en Colab, lo que facilita el acceso a los archivos y datos almacenados en Google Drive desde el entorno de Colab.

In [ ]:
from google.colab import drive

drive.mount("/content/gdrive")
!pwd

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content


Configura una sesión de Spark, carga los datos de un archivo CSV que contiene información sobre crímenes en Chicago

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.ml.feature import StringIndexer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import when, col
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import col, expr
from pyspark.sql import functions as F

spark = SparkSession.builder.appName("bigdata").getOrCreate()

pathCSV = '/content/gdrive/MyDrive/Cursos/IA95022/Big Data/Crimes.csv'

data = spark.read.csv(pathCSV, header=True, inferSchema=True)

In [ ]:
data.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: boolean (nullable = true)
 |-- Domestic: boolean (nullable = true)
 |-- Beat: integer (nullable = true)
 |-- District: integer (nullable = true)
 |-- Ward: integer (nullable = true)
 |-- Community Area: integer (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: integer (nullable = true)
 |-- Y Coordinate: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Location: string (nullable = true)



In [ ]:
data.show()

+--------+-----------+--------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|                Date|               Block|IUCR|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+--------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|11037294|   JA371270|03/18/2015 12:00:...|   0000X W WACKER DR|1153|DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|                BANK|

Se selecciona columnas específicas del DataFrame data, y crea una nueva columna llamada "Arrest_Num" que representa si hubo un arresto o no en función de la columna "Arrest." Si "Arrest" es igual a "true," se asigna el valor 1; de lo contrario, se asigna 0. Esto permite transformar la información sobre arrestos en una variable numérica y es útil para futuros análisis o la construcción de modelos de clasificación. Los resultados se muestran en el DataFrame resultante filtered_data.

In [ ]:
cols = ["Arrest",
        "Domestic",
        "Beat",
        "District",
        "Ward",
        "X Coordinate",
        "Y Coordinate",
        "Year"]

filtered_data = data.select(*cols, when(col("Arrest") == "true", 1).otherwise(0).alias("Arrest_Num"))
filtered_data.show()

+------+--------+----+--------+----+------------+------------+----+----------+
|Arrest|Domestic|Beat|District|Ward|X Coordinate|Y Coordinate|Year|Arrest_Num|
+------+--------+----+--------+----+------------+------------+----+----------+
| false|   false| 111|       1|  42|        null|        null|2015|         0|
| false|   false|2515|      25|  36|        null|        null|2018|         0|
| false|   false| 824|       8|  15|        null|        null|2016|         0|
| false|   false|1724|      17|  33|        null|        null|2018|         0|
| false|   false|2222|      22|  21|        null|        null|2014|         0|
| false|    true| 631|       6|   8|        null|        null|2018|         0|
| false|   false|2515|      25|  30|        null|        null|2018|         0|
| false|   false| 631|       6|   6|        null|        null|2018|         0|
| false|   false| 811|       8|  23|        null|        null|2015|         0|
| false|   false|1511|      15|  29|        null|   

In [ ]:
filtered_data.printSchema()

root
 |-- Arrest: boolean (nullable = true)
 |-- Domestic: boolean (nullable = true)
 |-- Beat: integer (nullable = true)
 |-- District: integer (nullable = true)
 |-- Ward: integer (nullable = true)
 |-- X Coordinate: integer (nullable = true)
 |-- Y Coordinate: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Arrest_Num: integer (nullable = false)



In [ ]:
filtered_data = filtered_data.na.drop()

Aquí se calcula la proporción de casos arrestados en un DataFrame, luego divide aleatoriamente los datos en conjuntos de entrenamiento y prueba para casos arrestados y no arrestados. Después, se combinan estos conjuntos de datos de entrenamiento y prueba, asegurando que mantengan una proporción similar de casos arrestados y no arrestados, lo que es fundamental en la preparación de datos para un modelo de clasificación.

In [ ]:
from pyspark.sql.functions import rand

arrested_count = filtered_data.filter(col("Arrest_Num") == 1).count()
total_count = filtered_data.count()
arrested_proportion = arrested_count / total_count

arrested_data = filtered_data.filter(col("Arrest_Num") == 1)
non_arrested_data = filtered_data.filter(col("Arrest_Num") == 0)

train_proportion = 0.7
train_arrested, test_arrested = arrested_data.randomSplit([train_proportion, 1 - train_proportion])
train_non_arrested, test_non_arrested = non_arrested_data.randomSplit([train_proportion, 1 - train_proportion])

train_data = train_arrested.union(train_non_arrested)
test_data = test_arrested.union(test_non_arrested)


In [ ]:
data = data.drop("Arrest")

Aquí se prepara las características, crea un modelo de regresión logística en Spark y lo entrena para predecir si un sospechoso fue arrestado o no. Utiliza un conjunto de características especificadas y las combina en una columna "features" utilizando VectorAssembler. Luego, el modelo de regresión logística se entrena en un conjunto de datos de entrenamiento y se utiliza para hacer predicciones en un conjunto de prueba. El resultado son predicciones que pueden evaluarse para determinar la precisión y eficacia del modelo en la clasificación de arrestos basados en las características proporcionadas.

In [ ]:
from pyspark.ml.feature import VectorAssembler

feature_cols = ["Domestic", "Beat", "District", "Ward", "X Coordinate", "Y Coordinate", "Year"]

assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
train_data = assembler.transform(train_data)
test_data = assembler.transform(test_data)

lr = LogisticRegression(labelCol="Arrest_Num")

# Entrena el modelo en el conjunto de entrenamiento
model = lr.fit(train_data)

# Realiza predicciones en el conjunto de prueba
predictions = model.transform(test_data)

Se encarga de evaluar el rendimiento del modelo de regresión logística. Utiliza dos evaluadores: uno para medir la precisión en una evaluación de clasificación binaria y otro para calcular el puntaje F1 en una evaluación de clasificación multiclase.

In [ ]:
evaluator = BinaryClassificationEvaluator(labelCol="Arrest_Num")
evaluator2 = MulticlassClassificationEvaluator(labelCol="Arrest_Num", predictionCol="prediction", metricName="f1")

accuracy = evaluator.evaluate(predictions)
f1_score = evaluator2.evaluate(predictions)

print(f"Accuracy: {accuracy}")
print("F1 Score:", f1_score)

Accuracy: 0.5953724943590049
F1 Score: 0.6351049520687025


Permite inspeccionar las predicciones del modelo en el conjunto de prueba y analizar cómo se comporta en casos específicos, lo que puede ser útil para comprender mejor su rendimiento.

In [ ]:
prediction_result = model.evaluate(test_data)
prediction_result.predictions.show()

+------+--------+----+--------+----+------------+------------+----+----------+--------------------+--------------------+--------------------+----------+
|Arrest|Domestic|Beat|District|Ward|X Coordinate|Y Coordinate|Year|Arrest_Num|            features|       rawPrediction|         probability|prediction|
+------+--------+----+--------+----+------------+------------+----+----------+--------------------+--------------------+--------------------+----------+
|  true|   false| 111|       1|   2|     1173162|     1899653|2005|         1|[0.0,111.0,1.0,2....|[0.62326070363783...|[0.65095978098321...|       0.0|
|  true|   false| 111|       1|   2|     1173177|     1899236|2005|         1|[0.0,111.0,1.0,2....|[0.62333241186021...|[0.65097607371411...|       0.0|
|  true|   false| 111|       1|   2|     1173179|     1899179|2006|         1|[0.0,111.0,1.0,2....|[0.66717666791062...|[0.66087067993912...|       0.0|
|  true|   false| 111|       1|   2|     1173895|     1899278|2005|         1|[0.0

In [ ]:
model.save("ArrestModel.dt")

In [ ]:
#spark.stop()